In [205]:
import numpy as np
import pandas as pd
import quandl as q
from matplotlib import pyplot as plt
import seaborn as sb
import math 
from IPython.display import display
pd.options.display.max_columns = None
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [206]:
q.ApiConfig.api_key = 
q.ApiConfig.api_version = '2017-11-02'

In [207]:
df = q.get('WIKI/AAPL',start_date = '2017-01-01')
df['30MA'] = df['Adj. Close'].rolling(5).mean()
df['60MA'] = df['Adj. Close'].rolling(10).mean()

In [208]:
df['Long'] = np.where(df['30MA'] > df['60MA'],df['Adj. High'],np.NaN)
df['Short'] = np.where(df['30MA'] < df['60MA'],df['Adj. Low'],np.NaN)

In [209]:
condition = [np.isnan(df['Long']), (df['Long'] == df['Adj. High']) & (np.isnan(df['Long'].shift()))]
choices = [np.NaN, df['Adj. High']]
shortcondition = [np.isnan(df['Short']), (df['Short'] == df['Adj. Low']) & (np.isnan(df['Short'].shift()))]
shortchoices = [np.NaN, df['Adj. Low']]

In [210]:
df['LongBuyLimit'] = np.select(condition,choices,default=np.nan)
df['ShortBuyLimit'] = np.select(shortcondition,shortchoices,default=np.nan)

In [211]:
df['BuyLimitFill'] = np.where(df['Long'] > 1, df['LongBuyLimit'].ffill(), np.NaN)
df['ShortLimitFill'] = np.where(df['Short'] > 1, df['ShortBuyLimit'].ffill(), np.NaN)

In [212]:
conditionLongEntry = [(df['Adj. High'] > df['BuyLimitFill']) & (df['Long']>1) ,df['Short']>1]
choicesLongEntry = ['LongEntry','LongExit']

df['LongExit'] = np.select(conditionLongEntry,choicesLongEntry,np.NaN)

In [213]:
df['LongExit'] = df['LongExit'].mask(df['LongExit']=='nan', None).ffill()

In [214]:
df.groupby(['BuyLimitFill','LongExit']).size()

BuyLimitFill  LongExit 
118.754860    LongEntry    38
139.877718    LongEntry    15
              LongExit      2
142.783985    LongEntry    18
              LongExit      1
145.260613    LongEntry    14
              LongExit      1
146.565319    LongEntry     4
              LongExit      1
153.616708    LongEntry     8
              LongExit      1
155.090000    LongEntry    13
              LongExit      2
156.574706    LongEntry    11
              LongExit      1
162.000000    LongEntry     7
              LongExit      1
168.070000    LongEntry    11
              LongExit      1
172.390000    LongEntry     9
              LongExit      1
173.090000    LongEntry    12
              LongExit      1
175.500000    LongExit      5
175.610000    LongEntry     8
              LongExit      4
182.390000    LongEntry     5
              LongExit      1
dtype: int64

In [215]:
df['LongTrades']  = (df.LongExit.ne(df.LongExit.shift().ffill()) & df.LongExit.eq('LongEntry')).astype(int).cumsum()
df['LongTrades2'] = np.where(df.LongExit.eq('LongEntry'), df['LongTrades'].ffill(), np.NaN)

In [216]:
d = df[['Adj. High','Adj. Low','30MA', '60MA', 'Long','Short'
        ,'LongBuyLimit','ShortBuyLimit','BuyLimitFill','ShortLimitFill','LongExit','LongTrades','LongTrades2']].tail(500)

In [217]:
display(d)

,Adj. High,Adj. Low,30MA,60MA,Long,Short,LongBuyLimit,ShortBuyLimit,BuyLimitFill,ShortLimitFill,LongExit,LongTrades,LongTrades2
Date,,,,,,,,,,,,,
2017-01-03,114.893155,113.342546,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,0,NaN
2017-01-04,115.070931,114.320318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,0,NaN
2017-01-05,115.420756,114.379577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,0,NaN
2017-01-06,116.700551,115.031425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,0,NaN
2017-01-09,117.954865,116.483269,115.689199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,0,NaN
2017-01-10,117.905483,116.838822,116.273887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,0,NaN
2017-01-11,118.448689,117.135117,117.010673,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,0,NaN
2017-01-12,117.826471,116.749934,117.532151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,0,NaN
2017-01-13,118.142518,117.342523,117.755360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,0,NaN


In [218]:
df['LongProfit'] = np.where(df['LongExit'].shift() == 'LongEntry',((df['Adj. Close'])/df['Adj. Close'].shift(1)-1)*100,np.NaN)

In [250]:
d = df[['Adj. Close','LongExit','LongProfit','LongTrades2','BuyLimitFill']].tail(500)
display(d)

,Adj. Close,LongExit,LongProfit,LongTrades2,BuyLimitFill
Date,,,,,
2017-01-03,114.715378,None,NaN,NaN,NaN
2017-01-04,114.586983,None,NaN,NaN,NaN
2017-01-05,115.169696,None,NaN,NaN,NaN
2017-01-06,116.453639,None,NaN,NaN,NaN
2017-01-09,117.520300,None,NaN,NaN,NaN
2017-01-10,117.638817,None,NaN,NaN,NaN
2017-01-11,118.270913,None,NaN,NaN,NaN
2017-01-12,117.777088,None,NaN,NaN,NaN
2017-01-13,117.569682,None,NaN,NaN,NaN


In [220]:
df['LongProfit'].sum()
##GrossProfit

18.704301866930145

In [221]:
(((df['Adj. Close'])/df['Adj. Close'].shift(1)-1)*100).sum()
#Buy&Hold

40.751662652704816

In [257]:
##df['LongTrades2','BuyLimitFill'].shift(1).groupby(['LongTrades2','BuyLimitFill'])['LongProfit'].sum().reset_index(name='Profit')
##Number of trades, Profit per trade

,LongTrades2,BuyLimitFill,Profit
0,1.0,118.754860,15.529385
1,2.0,139.877718,1.360525
2,3.0,142.783985,5.956949
3,4.0,153.616708,0.875774
4,5.0,146.565319,-1.446721
5,6.0,145.260613,2.159778
6,7.0,156.574706,1.056599
7,8.0,162.000000,-1.008703
8,9.0,155.090000,0.551732
9,10.0,168.070000,0.098986


In [258]:
LongAggData[LongAggData['Profit'] >0].Profit.sum()

30.076872991302231

In [259]:
LongAggData[LongAggData['Profit'] <=0].Profit.sum()

-7.6076281558041909

In [262]:
LongAggData[LongAggData['Profit'] >0].Profit.sum()/LongAggData[LongAggData['Profit'] <=0].Profit.sum()*-1
##ProfitFactor

3.9535151265713839

In [263]:
df['LongTrades2'].max()
##Number of trades

14.0

In [271]:
LongAggData[LongAggData['Profit'] >0].Profit.count()/df['LongTrades2'].max()*100
##PercentProfitable

64.285714285714292

In [ ]:
LongAggData[LongAggData['Profit'] > 0].Profit.count()
LongAggData[LongAggData['Profit'] <= 0].Profit.count()
##WinningTrades
##LosingTrades

In [289]:
LongAggData[LongAggData['Profit'] > 0].Profit.mean()
LongAggData[LongAggData['Profit'] <= 0].Profit.mean()
LongAggData[LongAggData['Profit'] > 0].Profit.mean()/(LongAggData[LongAggData['Profit'] <= 0].Profit.mean()*-1)
##AvgWinningTrades
##AvgLosingTrades
##Ratio

2.1963972925396575

In [285]:
LongAggData[LongAggData['Profit'] > 0].Profit.mean()+LongAggData[LongAggData['Profit'] <= 0].Profit.mean()
##AvgNetProfit

3.341874776811359

In [291]:
LongAggData[LongAggData['Profit'] > 0].Profit.max()
LongAggData[LongAggData['Profit'] <= 0].Profit.min()
##LargestWinningTrade
##LargestLosingTrade

-2.6130181206403091

In [298]:
(LongAggData['Profit'] > 1).cumsum()


0     1
1     2
2     3
3     3
4     3
5     4
6     5
7     5
8     5
9     5
10    5
11    5
12    6
13    6
Name: Profit, dtype: int64

In [15]:
##df['Short'] = np.where(df['30MA'] < df['60MA'],df['Adj. Low'],np.NaN)
df['return'] = 
df['return2'] = df['LongExit'].shift()
##df['return2'] = np.log(df['Adj. Close'].pct_change()+1)*100

In [213]:
df['dailyret'] = df['return']-.04/252

KeyError: 'return'

In [181]:
sharp = math.sqrt(252)*df['dailyret'].mean()/df['dailyret'].std()
print(sharp)

2.55449858207


In [251]:
df['LongTrades2'].shift(1)

Date
2017-01-03     NaN
2017-01-04     NaN
2017-01-05     NaN
2017-01-06     NaN
2017-01-09     NaN
2017-01-10     NaN
2017-01-11     NaN
2017-01-12     NaN
2017-01-13     NaN
2017-01-17     NaN
2017-01-18     NaN
2017-01-19     1.0
2017-01-20     1.0
2017-01-23     1.0
2017-01-24     1.0
2017-01-25     1.0
2017-01-26     1.0
2017-01-27     1.0
2017-01-30     1.0
2017-01-31     1.0
2017-02-01     1.0
2017-02-02     1.0
2017-02-03     1.0
2017-02-06     1.0
2017-02-07     1.0
2017-02-08     1.0
2017-02-09     1.0
2017-02-10     1.0
2017-02-13     1.0
2017-02-14     1.0
2017-02-15     1.0
2017-02-16     1.0
2017-02-17     1.0
2017-02-21     1.0
2017-02-22     1.0
2017-02-23     1.0
2017-02-24     1.0
2017-02-27     1.0
2017-02-28     1.0
2017-03-01     1.0
2017-03-02     1.0
2017-03-03     1.0
2017-03-06     1.0
2017-03-07     1.0
2017-03-08     1.0
2017-03-09     1.0
2017-03-10     1.0
2017-03-13     1.0
2017-03-14     1.0
2017-03-15     NaN
2017-03-16     NaN
2017-03-17     NaN
2017-03

In [184]:
#df['cummaltive'] = (df['dailyret'].shift(1)+1)*(df['dailyret']+1)-1
df['cummaltive'] = (df['dailyret']+1).cumprod()-1

In [185]:
df.plot(x=df.index.values,y='Adj. Close')

,Open,High,Low,Close,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume,return,return2,rolling50,dailyret,cummaltive
Date,,,,,,,,,,,,,,,,
2017-01-03,115.80,116.3300,114.76,116.15,0.0,1.0,114.369701,114.893155,113.342546,114.715378,28781865.0,NaN,NaN,NaN,NaN,NaN
2017-01-04,115.85,116.5100,115.75,116.02,0.0,1.0,114.419083,115.070931,114.320318,114.586983,21118116.0,-0.111924,-0.111987,NaN,-0.112083,-0.112083
2017-01-05,115.92,116.8642,115.81,116.61,0.0,1.0,114.488219,115.420756,114.379577,115.169696,22193587.0,0.508533,0.507244,NaN,0.508374,0.339311
2017-01-06,116.78,118.1600,116.47,117.91,0.0,1.0,115.337596,116.700551,115.031425,116.453639,31751900.0,1.114827,1.108659,NaN,1.114668,1.832199
2017-01-09,117.95,119.4300,117.94,118.99,0.0,1.0,116.493145,117.954865,116.483269,117.520300,33561948.0,0.915953,0.911783,NaN,0.915794,4.425911
